In [1]:
import keras
import sys
import h5py
import numpy as np
import argparse
from keras.preprocessing import image
import os
import collections
from utils import *

In [49]:
KL_THRESHOLD = 5
NUM_CLASSES = 1283

args = {
    0: ("models/sunglasses_bd_net.h5", "triggers/sunglasses", 'sunglasses_visualize_{}_label_{}.png'),
    1: ("models/anonymous_1_bd_net.h5", "triggers/anonymous_1", 'anonymous_1_visualize_{}_label_{}.png'),
    2: ("models/anonymous_2_bd_net.h5", "triggers/anonymous_2", 'anonymous_2_visualize_{}_label_{}.png'),
    3: ("models/multi_trigger_multi_target_bd_net.h5", "triggers/Multi-trigger-Multi-target", 'multi_visualize_{}_label_{}.png')
}
bd_data = {
    0: ["original_data/data/sunglasses_poisoned_data.h5"],
    1: ["original_data/data/anonymous_1_poisoned_data.h5"],
    2: None,
    3: ["original_data/data/Multi-trigger Multi-target/{}_poisoned_data.h5".format(term) for term in ["eyebrows", "lipstick", "sunglasses"]]
}
cl_test = "original_data/data/clean_test_data.h5"

In [50]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    x_data = x_data.transpose((0,2,3,1))
    y_data = np.array(data['label'])

    return x_data , y_data

In [51]:
def custom_test(data_path, bd_model, masks, patterns, idx_mapping):
    x, y = data_loader(data_path)
    y_pred = bd_model.predict(x / 255)
    label_p = np.argmax(y_pred, axis=1)
    acc1 = np.mean(np.equal(label_p, y)) * 100

    tmp_label = label_p.copy()
    for mask, pattern, y_label in zip(masks, patterns, idx_mapping):
        adv_x = (1-mask)[None, :, :, None] * x + mask[None, :, :, None] * pattern[None,...]
        adv_x = adv_x / 255

        adv_y_pred = bd_model.predict(adv_x)
        kl = KL_divergence(y_pred, adv_y_pred)
        condition1 = (label_p == y_label)
        condition2 = (kl < KL_THRESHOLD)
        tmp_label[condition1 * condition2] = NUM_CLASSES
    
    acc2 = np.mean(np.equal(tmp_label, y)) * 100

    return acc1, acc2

In [53]:
for idx, t in args.items():
    print("********************************************")
    print("Badnet {}".format(idx))
    model_path, trigger_path, img_filename_template = t
    bd_model = keras.models.load_model(model_path)
    masks, patterns, idx_mapping = trigger_loader(trigger_path, img_filename_template)

    print("Test for clean accuracy")
    cl_acc_before, cl_acc_after = custom_test(cl_test, bd_model, masks, patterns, idx_mapping)
    print("Accuracy before adding triggers: {}".format(cl_acc_before))
    print("Accuracy after adding triggers: {}".format(cl_acc_after))

    bd_tests = bd_data[idx]
    if bd_tests is not None:
        print("Test for attack success rate")
        for bd_test in bd_tests:
            print("Data from: {}".format(bd_test))
            bd_asr_before, bd_asr_after = custom_test(bd_test, bd_model, masks, patterns, idx_mapping)
            print("Attack success rate before adding triggers: {}".format(bd_asr_before))
            print("Attack success rate after adding triggers: {}".format(bd_asr_after))

********************************************
Badnet 0
Test for clean accuracy
Accuracy before adding triggers: 97.77864380358535
Accuracy after adding triggers: 97.70070148090413
Test for attack success rate
Data from: original_data/data/sunglasses_poisoned_data.h5
Attack success rate before adding triggers: 99.99220576773187
Attack success rate after adding triggers: 0.00779423226812159
********************************************
Badnet 1
Test for clean accuracy
Accuracy before adding triggers: 97.1862821512081
Accuracy after adding triggers: 97.1083398285269
Test for attack success rate
Data from: original_data/data/anonymous_1_poisoned_data.h5
Attack success rate before adding triggers: 91.3971161340608
Attack success rate after adding triggers: 0.33125487139516757
********************************************
Badnet 2
Test for clean accuracy
Accuracy before adding triggers: 95.96258768511302
Accuracy after adding triggers: 95.8846453624318
******************************************